In [15]:
pip install geopy


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [79]:
import pandas as pd
import json
from geopy.geocoders import Nominatim
from tqdm.auto import tqdm
import overpy


api = overpy.Overpass()

geolocator = Nominatim(user_agent="gtfs_processor", timeout=2000)

with open('university_osm.json') as fp:
    data = json.load(fp)

# df = pd.read_json('poi.json')
data

df = []
for item in tqdm(data['elements']):

    new_json_data = {}
    if item['type']  == 'node':
    
        
        location = geolocator.reverse((item['lat'], item['lon']), exactly_one=True)
        if location and 'osm_id' in location.raw:
            osm_id =location.raw['osm_id']
        else:
            osm_id = 'na'
        
        new_json_data = {
            "latitude": item['lat'],
            "longitude": item['lon'],
            "name": item['tags']['name'],
            "type": item['tags']['amenity'],
            "osm_id": osm_id,
        }
        df.append(new_json_data)

    
    else:
        if 'nodes' in item:
            if 'name' in item['tags']:
                node_id = item['nodes'][0]
                query = f"""
                [out:json];
                node({node_id});
                out body;
                """
                result = api.query(quer
                                   y)
                node = result.nodes[0]

                
                latitude = float(node.lat)
                longitude = float(node.lon)
                
                location = geolocator.reverse((latitude, longitude), exactly_one=True)
                if location and 'osm_id' in location.raw:
                    osm_id =location.raw['osm_id']
                else:
                    osm_id = 'na'
                    
                if 'amenity' in item['tags']:
                    type = item['tags']['amenity']
                elif 'building' in item['tags']:
                    type = item['tags']['building']
                else:
                    type = 'university'
                    
                name = ''
                if 'alt_name' in item['tags']:
                    name = ", ".join([item['tags']['alt_name'], item['tags']['name']])
                else:
                    name = item['tags']['name']
                    
                new_json_data = {
                    "latitude": node.lat,
                    "longitude": node.lon,
                    "name": name,
                    "type": type,
                    "osm_id": osm_id,
                }
                df.append(new_json_data)

df = pd.DataFrame(df)
df

df = pd.DataFrame(df).to_csv('poi_and_osm_university.csv', index=False)

newdf = pd.read_csv('poi_and_osm_university.csv')

print(newdf.tail())

  0%|          | 0/77 [00:00<?, ?it/s]

     latitude   longitude                                               name  \
50  46.067655   11.149313     Povo 2, Dipartimento di Ingegneria Industriale   
51  46.066658   11.119793       Dipartimento di Sociologia e Ricerca Sociale   
52  46.067040   11.138900  Dipartimento di Ingegneria Civile, Ambientale ...   
53   8.043931  126.063627     Agusan Del Sur State University - Trento Annex   
54  46.069701   11.121070                      Biblioteca Comunale di Trento   

          type       osm_id  
50  university  11726705877  
51  university    930522309  
52  university   7699915777  
53  university   1016831603  
54     library   1181361605  


In [80]:

newdf = pd.read_csv('poi_and_osm_lib.csv')
print(newdf.tail())

     latitude   longitude                                               name  \
50  46.067655   11.149313     Povo 2, Dipartimento di Ingegneria Industriale   
51  46.066658   11.119793       Dipartimento di Sociologia e Ricerca Sociale   
52  46.067040   11.138900  Dipartimento di Ingegneria Civile, Ambientale ...   
53   8.043931  126.063627     Agusan Del Sur State University - Trento Annex   
54  46.069701   11.121070                      Biblioteca Comunale di Trento   

          type       osm_id  
50  university  11726705877  
51  university    930522309  
52  university   7699915777  
53  university   1016831603  
54     library   1181361605  


In [ ]:

# df1 = pd.read_csv('poi_and_osm.csv')
# df1 = df1.drop(columns=['id'])
# df1 = pd.DataFrame(df1).to_csv('poi_and_osm_new.csv', index=False)
# df1 = pd.read_csv('poi_and_osm_new.csv')
# print(df1.head())

In [4]:
import pandas as pd

df = pd.read_csv('poi_and_osm_full.csv')  

column_to_move = 'osm_id'

columns = [column_to_move] + [col for col in df.columns if col != column_to_move]
df = df[columns]

df.to_csv('poi_and_osm_full_1.csv', index=False)


In [ ]:

def convert_value(value):
    
    try:
        if '.' in value:
            return float(value)
        elif len(value) > 10:
                return value
        else:
            return int(value)
    except ValueError:
        return value 

def csv_to_json(csv_file, json_file):
    
    with open(csv_file, mode='r') as f:
        reader = csv.DictReader(f)
        rows = []
        for row in reader:
            converted_row = {key: convert_value(value) for key, value in row.items()}
            rows.append(converted_row)

    with open(json_file, mode='w') as f:
        json.dump(rows, f, indent=4)

csv_file = "../auxilary datas/poi_and_osm_full.csv"
json_file = "../auxilary datas/poi_and_osm_full1.json"
csv_to_json(csv_file, json_file)


In [ ]:
# Pizzeria - pizzeria

# gelateria - gelateria-pasticceria
# restaurant - ristorante

# Biblioteca - biblioteca
# library - biblioteca

# Bar - bar
# Birreria / Pub birreria

# Supermercato/minimarket - supermercato

# Farmacia - farmacia

# teatro/opera/cinema - cinema

# appartamentovacanze - appartamento

# appartamenti vacanze - appartamento

# wifi - internet-point
# student_accommodation - student-accommodation

# university - university-facility

# fast_food - fast-food